<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Tomás Fontecilla </em><br>

</div>

# Machine Learning Avanzado
*02 de Diciembre de 2024*

#### Integrantes: 
` Gabriel Álvarez - Rosario Valderrama `

## 1. Objetivo

El objetivo de este informe es ajustar **redes neuronales convolucionales** para realizar un análisis de ciencia de datos utilizando la base extraída de Kaggle ` chihuahuas vs muffins `. Luego, compararemos los resultados con un **modelo de perceptrón multicapa**.

A ambos modelos se aplicará 3 estructuras diferentes: 
- Regularización L2
- Dropout
- Decay

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, Sequential, regularizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay

## 2. Introducción

A través del dataset ` chihuahuas vs muffins ` que son imágenes, se busca entrenar un modelo de clasificación de redes neuronales (perceptrón multicapa y convolucionales) para lograr clasificar correctamente cada imagen. Para esto, los pasos a seguir son:

1. Cargar las imágenes
2. Preprocesar las imágenes
3. Crear el modelo
4. Entrenar el modelo
5. Evaluar el modelo
6. Guardar el modelo
7. Hacer predicciones

## 3. Metodología

### 3.1 Perceptrón Multicapa (MLP)

Para comenzar, se carga las imágenes desde las carpetas data/train y data/test, donde se infieren las etiquetas de ellas basándose en los nombres de las subcarpetas. Todas las imágenes se redimensionan a 128 x 128 para asegurar que todas tengan el mismo tamaño, para luego agruparlas en lotes de 32 imágenes, lo que permite poder procesarlas de forma más eficiente. Todas las imágenes son "barajadas" del conjunto de entrenamiento, para así evitar patrones en el orden de los datos que finalmente afecte este entrenamiento.

Se normalizan los valores de cada pixel de las imágenes, dividiéndolos por 255, para que así estén dentro del rango de 0 a 1. Si no se aplicara esta normalización, los valores podrían estar en el rango del 0 a 255, dificultando así la convergencia del modelo.

In [2]:


# Directorios
train_dir = "data/train"
test_dir = "data/test"

# 1. Cargar las imágenes
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),  # Tamaño fijo
    batch_size=32,
    shuffle=True
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)

# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

# Normalización (valores entre 0 y 1)
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


Dado a que el perceptrón multicapa no puede procesar las imágenes en 2D, requiere que las imágenes se transformen en vectores 1D. Para esto, se busca aplanar las imágenes, convirtiendo cada imagen de 128 x 128 x 3 (RGB) en un vector 1D de tamaño 49152.

In [30]:
# Aplanar las imágenes para el MLP
# Convierte imágenes 2D en vectores 1D
def flatten_images(x, y):
    flat_dim = 128 * 128 * 3  # Tamaño fijo de las imágenes aplanadas
    x = tf.reshape(x, [tf.shape(x)[0], flat_dim])  # Garantiza que sea un tensor válido
    return x, y

train_dataset_flat = train_dataset.map(flatten_images)
test_dataset_flat = test_dataset.map(flatten_images)


El modelo del perceptrón multicapa se construye utilizando la API **Sequiential** de TensorFlow/Keras, lo que permite definir una arquitectura de red neuronal de manera secuencial, que va atravesando capa por capa.

La primera capa, llamada Input, especifica la forma de las entradas del modelo, que corresponde a vectores del tamaño 128 x 128 x 3 = 49152, resultado de aplanar las imágenes RGB de entrada que previamente se redimencionaron a 128 x 128 pixeles. 

Posteriormente, se definen 3 capas ocultas densas llamadas **dense**, que están completamente conectadas. Esto significa que cada neurona de una capa está conectada a todas las neuronas de la siguiente capa. La primera capa oculta contiene 256 neuronas, la segunda 128 neuronas y la tercera 64 neuronas.

Todas las capas ocultas usan la función de activación ReLU,que se selecciona por su capacidad para introducir no linealidad en la red, permitiendo que el modelo aprenda relaciones complejas entre los datos de entrada y los de salida. Esta función activa únicamente valores positivos (los deja sin cambios) y transforma los valores negativos a cero. Esto mejora la eficiencia computacional en comparación con otras funciones de activación y ayuda a evitar problemas como el desvanecimiento del gradiente, que puede ralentizar el aprendizaje en redes profundas. Al activar solo ciertas neuronas según las entradas, ReLU también ayuda a que la red se enfoque en características relevantes, optimizando su capacidad de generalización.

Finalmente, la capa de salida también es densa y tiene tantas neuronas como clases en el problema, lo cual se especifica como len(class_names). En este caso, las clases corresponden a las categorías de las imágenes (chihuahuas y muffins). Esta capa utiliza la función de activación softmax, que convierte las salidas en probabilidades normalizadas, asignando una probabilidad a cada clase. Se utiliza esta activación ya que permite interpretar la salida del modelo como la probabilidad de que una imagen pertenezca a cada clase.

En conjunto, estas capas y funciones de activación permiten que el modelo procese los datos de entrada, aprenda patrones significativos en las imágenes y genere predicciones probabilísticas sobre las clases a las que pertenece cada imagen. 

In [31]:
# Crear el modelo MLP
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),  # Imágenes aplanadas (128x128x3)
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

**mlp_model.compile** configura el modelo de Perceptrón Multicapa para el entrenamiento, definiendo tres componentes clave: el optimizador, la función de pérdida y las métricas de evaluación. El optimizador Adam se utiliza por su capacidad para ajustar dinámicamente la tasa de aprendizaje, combinando las ventajas de métodos como Momentum y RMSProp. La función de pérdida sparse_categorical_crossentropy es adecuada para problemas de clasificación multiclase con etiquetas enteras, como las clases inferidas de las imágenes. Finalmente, se especifica la métrica de exactitud (accuracy), que mide la proporción de predicciones correctas durante el entrenamiento y la evaluación, proporcionando una evaluación clara del desempeño del modelo.

In [32]:
# Compilar el modelo
mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_32 (Dense)                │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

**mlp_model.fit** se utiliza para entrenar el modelo con los datos del conjunto de entrenamiento (train_dataset_flat). La cantidad de "epochs" se define mediante el parámetro epochs=10, indicando que el modelo realizará 10 iteraciones completas sobre todo el conjunto de datos de entrenamiento. Durante cada "epoch", el modelo ajusta sus pesos en función de los gradientes calculados para minimizar la función de pérdida. Cada pasada completa permite al modelo mejorar su capacidad para aprender los patrones en los datos, mientras que un número excesivo de "epochs" podría llevar al sobreajuste, donde el modelo se adapta demasiado a los datos de entrenamiento y pierde capacidad de generalización para datos nuevos. 

In [33]:
# Entrenar el modelo
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 19s 122ms/step - accuracy: 0.5669 - loss: 3.0801
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 17s 113ms/step - accuracy: 0.6124 - loss: 1.0480
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 121ms/step - accuracy: 0.6734 - loss: 0.6690
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 17s 116ms/step - accuracy: 0.7087 - loss: 0.5962
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 111ms/step - accuracy: 0.7560 - loss: 0.5035
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 17s 115ms/step - accuracy: 0.7711 - loss: 0.4816
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 119ms/step - accuracy: 0.7726 - loss: 0.4757
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 17s 113ms/step - accuracy: 0.7831 - loss: 0.4682
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 111ms/step - accuracy: 0.7844 - loss: 0.4516
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 18s 118ms/step - accuracy: 0.7864 - loss: 0.4558


**mlp_model.evaluate** permite evaluar el desempeño del modelo en el conjunto de datos de prueba (test_dataset_flat). Este proceso calcula la pérdida, que mide qué tan bien o mal se comporta el modelo con datos nuevos, y el accuracy, que indica el porcentaje de predicciones correctas. 

La pérdida ayuda a diagnosticar posibles problemas en el ajuste del modelo, mientras que el accuracy refleja su capacidad de generalización y su desempeño al momento de clasificar las imágenes de salida. 

In [34]:
# Evaluar el modelo
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida del MLP: {loss}")
print(f"Exactitud del MLP: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8445 - loss: 0.4156
Pérdida del MLP: 0.5101656317710876
Exactitud del MLP: 0.7719594836235046


In [14]:
# Crear el modelo MLP con regularización
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),  # Imágenes aplanadas (128x128x3)
    
    # Primera capa densa con regularización L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Segunda capa densa con regularización L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Tercera capa densa con regularización L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Compilar el modelo
mlp_model.compile(
    optimizer='adam',  # Puedes añadir decay si lo deseas
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Entrenar el modelo
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 154ms/step - accuracy: 0.5278 - loss: 10.7587
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 149ms/step - accuracy: 0.5065 - loss: 2.6075
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.4946 - loss: 2.0486
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - accuracy: 0.5226 - loss: 1.8636
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.5236 - loss: 1.7364
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 141ms/step - accuracy: 0.5078 - loss: 1.6250
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 141ms/step - accuracy: 0.5239 - loss: 1.5448
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 41s 143ms/step - accuracy: 0.5304 - loss: 1.4312
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 148ms/step - accuracy: 0.5145 - loss: 1.3634
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 147ms/step - accuracy: 0.5334 - loss: 1.3081


In [ ]:
# Evaluar el modelo
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida del MLP con regularización y dropout: {loss}")
print(f"Exactitud del MLP con regularización y dropout: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.7143 - loss: 1.2053
Pérdida del MLP: 1.2264331579208374
Exactitud del MLP: 0.5067567825317383


#### 3.1.1 Perceptrón Multicapa, aplicando técnicas de regularización para prevenir el sobreajuste.

Para prevenir el sobreajuste en este modelo, se recomienda utilizar una combinación de tres técnicas de regularización: Dropout, Regularización L2, y Decay. 

**Dropout** introduce robustez al modelo al apagar aleatoriamente un porcentaje de las neuronas durante el entrenamiento, obligando al modelo a aprender patrones más generalizados en lugar de depender de combinaciones específicas de pesos. Esto se implementa añadiendo capas de Dropout con un porcentaje, como layers.Dropout(0.3) para desactivar el 30% de las neuronas en cada capa. 

**Regularización L2**, penaliza los pesos grandes al añadir una restricción basada en la suma de los cuadrados de los pesos, lo que fomenta que el modelo mantenga distribuciones de pesos más balanceadas. Se configura con el argumento kernel_regularizer=regularizers.l2(0.01) en las capas densas.

**Decay** ajusta dinámicamente la tasa de aprendizaje del optimizador, permitiendo pasos grandes al inicio para aprender rápidamente y pasos más pequeños al final para refinar los pesos; esto se implementa mediante un programa de decaimiento como ExponentialDecay. 

Estas técnicas ayudan a mejorar la generalización del modelo al reducir la complejidad, distribuir el aprendizaje de manera uniforme y evitar la dependencia excesiva en subconjuntos específicos de parámetros.

In [27]:
# Decaimiento de la tasa de aprendizaje
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=0.01,  # Tasa de aprendizaje inicial
    decay_steps=1000,           # Número de pasos para aplicar el decay
    decay_rate=0.9,             # Factor de decaimiento
    staircase=False             # Decaimiento continuo
)

# Crear el modelo con Dropout y Regularización L2
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),  # Entrada aplanada de las imágenes

    # Primera capa densa con L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Segunda capa densa con L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Tercera capa densa con L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Compilar el modelo con Decay
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

mlp_model.compile(
    optimizer=optimizer,  # Optimización con Decay
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

**- Decay:** El optimizador Adam se configura con una tasa de aprendizaje dinámica. Ésto reduce gradualmente la tasa de aprendizaje a medida que avanza el entrenamiento.

**- Dropout:** Se añade después de las capas densas para apagar aleatoriamente el 30% de las neuronas en cada paso. 

**- Regularización L2:** Añade una penalización a los pesos grandes en las capas densas. 



In [28]:
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 153ms/step - accuracy: 0.5038 - loss: 67.7531
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 23s 153ms/step - accuracy: 0.5030 - loss: 2.4666
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 158ms/step - accuracy: 0.5332 - loss: 2.0428
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 23s 155ms/step - accuracy: 0.5385 - loss: 2.0125
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 22s 151ms/step - accuracy: 0.5375 - loss: 1.6819
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - accuracy: 0.5287 - loss: 1.4472
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - accuracy: 0.5208 - loss: 2.1587
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 143ms/step - accuracy: 0.5220 - loss: 3.1531
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.5324 - loss: 1.3741
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 144ms/step - accuracy: 0.5345 - loss: 1.2125


In [29]:
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida del MLP con regularización, dropout y tasa de aprendizaje: {loss}")
print(f"Exactitud del MLP con regularización, dropout y tasa de aprendizaje: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.8584 - loss: 1.6098
Pérdida del MLP con regularización, dropout y tasa de aprendizaje: 1.6491082906723022
Exactitud del MLP con regularización, dropout y tasa de aprendizaje: 0.5405405163764954


### 3.2 Redes convolucionales

A continuación se presentan tres modelos de redes neuronales convolucionales, cada uno con una estructura diferente. Su estructura varía en la combinación de tres técnicas de regularización: Dropout, Regularización L2, y Decay.

Cabe mencionar que para hacer comparables los resultados, la fase de capa completamente conectada corresponde a la misma estructura de los tres modelos de perceptrón multicapa, sólo agregando las capas convolucionales y de pooling con combinaciones de las técnicas de regularización mencionadas.

In [4]:
model_cnn = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Dropout en la primera capa convolucional

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Compilar el modelo
model_cnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model_cnn.summary()




c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
# Entrenar el modelo
model_cnn.fit(train_dataset, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 61s 394ms/step - accuracy: 0.7150 - loss: 1.8823
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 392ms/step - accuracy: 0.8515 - loss: 0.3693
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 393ms/step - accuracy: 0.8791 - loss: 0.2985
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 388ms/step - accuracy: 0.8768 - loss: 0.3070
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 390ms/step - accuracy: 0.9138 - loss: 0.2192
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 400ms/step - accuracy: 0.9357 - loss: 0.1791
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 391ms/step - accuracy: 0.9477 - loss: 0.1350
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 392ms/step - accuracy: 0.9627 - loss: 0.0963
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 389ms/step - accuracy: 0.9536 - loss: 0.1418
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 388ms/step - accuracy: 0.9742 - loss: 0.0728


In [6]:
# Evaluar el modelo
loss, accuracy = model_cnn.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - accuracy: 0.9111 - loss: 0.3791
Pérdida: 0.45413675904273987
Exactitud: 0.8876689076423645


A continuación, se crea el modelo CNN 

In [ ]:
# A continuación, se crea el modelo CNN con regularización L2 y Dropout en las capas densas.
model_cnn2 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    # Primera capa densa con regularización L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Segunda capa densa con regularización L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Tercera capa densa con regularización L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model_cnn2.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_cnn2.summary()



c:\Magister\3er trimestre\MLA\MLA_T2\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [8]:
# Entrenar el modelo
model_cnn2.fit(train_dataset, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 380ms/step - accuracy: 0.7169 - loss: 8.7107
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 56s 379ms/step - accuracy: 0.8246 - loss: 3.9321
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 388ms/step - accuracy: 0.8377 - loss: 2.5537
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 387ms/step - accuracy: 0.8636 - loss: 1.8992
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 386ms/step - accuracy: 0.8910 - loss: 1.5189
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 389ms/step - accuracy: 0.8956 - loss: 1.2643
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 388ms/step - accuracy: 0.9075 - loss: 1.0698
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 387ms/step - accuracy: 0.8993 - loss: 0.9594
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 58s 388ms/step - accuracy: 0.9151 - loss: 0.8149
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 57s 387ms/step - accuracy: 0.9197 - loss: 0.7452


In [9]:
# Evaluar el modelo
loss, accuracy = model_cnn2.evaluate(test_dataset)

print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.9234 - loss: 0.6687
Pérdida: 0.8412922620773315
Exactitud: 0.8277027010917664


A continuación, se crea el modelo CNN con regularización L2 y Dropout en las capas de convolución y densas. Además, al optimizador se le añade un decaimiento exponencial de la tasa de aprendizaje.

In [ ]:
# Decaimiento de la tasa de aprendizaje
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=0.01,  # Tasa de aprendizaje inicial
    decay_steps=1000,           # Número de pasos para aplicar el decay
    decay_rate=0.9,             # Factor de decaimiento
    staircase=False             # Decaimiento continuo
)
model_cnn3 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),  # Dropout en las capas convolucionales

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    # Primera capa densa con L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Segunda capa densa con L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Tercera capa densa con L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Optimizer con decay
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)
model_cnn3.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model_cnn3.summary()



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 126, 126, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 61, 61, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,558,210 (25.02 MB)

 Trainable params: 6,557,762 (25.02 MB)

 Non-trainable params: 448 (1.75 KB)

In [11]:
# Entrenar el modelo
model_cnn3.fit(train_dataset, epochs=10)



Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 61s 398ms/step - accuracy: 0.6488 - loss: 46.7220
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 396ms/step - accuracy: 0.7590 - loss: 2.7234
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 59s 396ms/step - accuracy: 0.7893 - loss: 1.9580
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.7988 - loss: 1.5279
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 403ms/step - accuracy: 0.8135 - loss: 1.3070
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 404ms/step - accuracy: 0.8155 - loss: 1.1506
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 406ms/step - accuracy: 0.8178 - loss: 1.1410
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.8199 - loss: 1.0511
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.8316 - loss: 1.0670
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 60s 405ms/step - accuracy: 0.8357 - loss: 1.0282


In [12]:
# Evaluar el modelo
loss, accuracy = model_cnn3.evaluate(test_dataset)

print(f"Pérdida: {loss}")

print(f"Exactitud: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.4453 - loss: 1.8645
Pérdida: 1.4726265668869019
Exactitud: 0.6494932174682617


## Comparación de resultados

| Modelo   | Accuracy FNN | Loss FNN | Accuracy CNN | Loss CNN |
|----------|--------------|----------|--------------|----------|
| Modelo 1                                          | 0.51         | 0.77     | 0.88         | 0.45     |
| Modelo 2 Dropout y regularización l2              | 0.50         | 1.22     | 0.82         | 0.84     |
| Modelo 3 Dropout, regularización y learning rate  | 0.54         | 1.64     | 0.64         | 1.47     |

Como se puede observar en la tabla comparativa, el modelo de redes neuronales convolucionales (CNN) obtiene mejores resultados en términos de accuracy y loss **en todos los modelos** en comparación con el modelo de perceptrón multicapa (FNN).

Por su parte, la pérdida en los FNN y en CNN es mayor en los modelos con modificaciones, lo que podría indicar que estos cambios pueden estar afectando la capacidad de aprendizaje del modelo.

En cuanto al tercer modelo con CNN se puede observar una disminución en el accuracy y un aumento en la pérdida, lo que podría indicar que la inserción de la técnica de decaimiento de la tasa de aprendizaje no está siendo efectiva en este caso.

Cabe destacar que en las ejecuciones se pudo observar que las FNN tomaron menos tiempo en ser entrenadas en comparación con las CNN, lo que podría deberse a la mayor complejidad de las CNN y a la cantidad de capas que estas poseen.



## Conclusiones

En relación al desempeño de los modelos FNN, se observa que todos tienen un accuracy similar cercano al 50%. Debido a que la cantidad de categorías a predecir son 2, un accuracy de 50% es equivalente a un modelo que predice al azar, lo que sugiere que el modelo no está aprendiendo patrones significativos en los datos de entrada.

Por su parte, el desempeño de los modelos CNN es significativamente mejor, con accuracies superiores al 60% en todos los casos y alcanzando un 88% para el primer modelo. Esto sugiere que las redes neuronales convolucionales son más efectivas para aprender patrones en imágenes y clasificarlas correctamente, en comparación con los perceptrones multicapa.



Para la comparación de ambos modelos:
- Desempeño: ¿Qué modelo tiene mayor precisión?
- Capacidad de generalización: ¿Cuál tiene menor pérdida en el conjunto de prueba?
- Velocidad de entrenamiento: ¿Cuál fue más rápido?